In [135]:
!pip3 install torch torchvision pandas numpy pytorch_lightning
import numpy as np
import pandas as pd
import torch 
import pickle 
np.random.seed(420)

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [136]:
# !wget -nc https://files.grouplens.org/datasets/movielens/ml-latest-small.zip 
# !unzip -n ./ml-latest-small.zip
ratings = pd.read_csv('ml-latest-small/ratings.csv', parse_dates=['timestamp'])
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [137]:
def encode_column(column):
    """ Encodes a pandas column with continous IDs"""
    keys = column.unique()
    key_to_id = {key:idx for idx,key in enumerate(keys)}
    return key_to_id, np.array([key_to_id[x] for x in column]), len(keys)

def encode_df(data):
    """Encodes rating data with continuous user and movie ids"""
    
    movie_ids, data['movieId'], num_movies = encode_column(data['movieId'])
    user_ids, data['userId'], num_users = encode_column(data['userId'])
    return data, num_users, num_movies, user_ids, movie_ids


ratings, num_users, num_items, user_ids, movie_ids = encode_df(ratings)
print("Number of users :", num_users)
print("Number of movies :", num_items)

ratings.head()

Number of users : 610
Number of movies : 9724


,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224
3,0,3,5.0,964983815
4,0,4,5.0,964982931


In [138]:
ratings['rank'] = ratings.groupby(['userId'])['timestamp'].rank(method = 'first', ascending=False, pct=True)

#taking the most recent 20% of the ratings of all users as test set 
train_ratings = ratings[ratings['rank'] > .2]
test_ratings = ratings[ratings['rank'] <= .2]

train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId','movieId','rating']]

print('train set size :', len(train_ratings))
print('test set size :', len(test_ratings))
print('total set size :', len(test_ratings)+len(train_ratings))

train set size : 80896
test set size : 19940
total set size : 100836


In [139]:
# Run to biuld Model
#=========================================================================================================

from dnn_model import DNN
import pytorch_lightning as pl
model = DNN(num_users, num_items, train_ratings)
trainer = pl.Trainer(max_epochs=10, enable_progress_bar=True, logger=False, checkpoint_callback=False)
trainer.fit(model)

filename = 'dnn_model.sav'
# To save model when it is built 

pickle.dump(model, open(filename, 'wb'))

#=========================================================================================================

/usr/local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 39.0 K
1 | item_embedding | Embedding | 622 K 
2 | fc1            | Linear    | 8.3 K 
3 | fc2            | Linear    | 2.1 K 
4 | fc3            | Linear    | 528   
5 | fc4            | Linear    | 136   
6 | fc5            | Linear    | 36    
7 | fc6            | Linear    | 10    
8 | output         | Linear    | 3     
9 | drop           | Dropout   | 0     
---------------------------------------------
672 K 

Training: 0it [00:00, ?it/s]

In [140]:
test_set = set(zip(test_ratings['userId'], test_ratings['movieId'], test_ratings['rating']))


import math

def normal_round(n):
    dec = n - math.floor(n)
    if dec < 0.75 and dec >.25:
      return n - dec + .5

    elif dec < .5:
      return math.floor(n)
    
    return math.ceil(n)


model = pickle.load(open(filename, 'rb'))
predictions = []
rmse_sq = 0 
for (u,i,r) in test_set:
    
    pred = model(torch.tensor([u]), torch.tensor([i]))
    pred = normal_round(pred.item())
    rmse_sq+= ((pred - r)*(pred - r))/len(test_set)

rmse = math.sqrt(rmse_sq)
print('MSE : ', rmse_sq)
print('RMSE : ', rmse)


MSE :  1.078096790371057
RMSE :  1.038314398614917


In [142]:
# Mapping of encoded userIds with source data saved externally 
pickle.dump(movie_ids, open('movie_ids_map.pkl', 'wb'))
pickle.dump(user_ids, open('user_ids_map.pkl', 'wb'))

def read_movies_data():
  names = ['movieId', 'title', 'genres']
  return pd.read_csv('ml-latest-small/movies.csv', sep=',', names=names, engine='python')

data = read_movies_data()
data = data.drop([0])
data.head(5)

movies = set(zip(data['movieId'], data['title']))
movie_dict = {int(m[0]): m[1]  for m in movies}
pickle.dump(movie_dict, open('movie_dict.pkl', 'wb'))

